# runAnalysis( )

저번 챕터와는 다르게 `idxReview`의 길이는 198행 전체를 뜻하는게 아니고, dataReviewTesting의 행 따라 달라진다.

In [ ]:
    # idxReview번째 리뷰를 가져와 분석한다. 감정의 예측값과 실제값을 비교해 결과를 리턴.
    def runAnalysis(self, idxReview):
        probLogPositive = 0
        probLogNegative = 0
        idxUsedWords, usedWords = self.findUsedWords(idxReview)    # idxReview번째 리뷰에 사용된 단어들과 그 단어들의 인덱스.                                

        # 불러온 각각의 단어의 긍정/부정 확률을 모두 더해 초기화한 변수에 저장.
        for i in range(len(idxUsedWords)):    # idxReview번째 리뷰에 사용된 단어들만큼 for loop.
            idxWord = idxUsedWords[i]
            positive, negative = self.calculateProbWord(idxWord)    # 각각의 단어들마다 긍정/부정일 확률 계산.
            probLogPositive = probLogPositive + np.log(positive)
            probLogNegative = probLogNegative + np.log(negative)

        positiveProb1, negativeProb1 = self.calculateProbReview()
        probLogPositive = probLogPositive + np.log(positiveProb1)
        probLogNegative = probLogNegative + np.log(negativeProb1)

        # (긍정이든 부정이든)감정의 예측값과 감정의 실제값이 같다면 correct = 1, 아니면 correct = 0.
        if self.dataReviewTesting[idxReview] == 1:
            if  probLogPositive > probLogNegative:
                correct = 1
            else:
                correct = 0
        else:
            if probLogNegative < probLogPositive:
                correct = 0
            else:
                correct = 1
                
        return probLogPositive, probLogNegative, correct

In [ ]:
    def runAnalysis(self, idxReview):
        probLogPositive = 0
        probLogNegative = 0
        idxUsedWords, usedWords = self.findUsedWords(idxReview)

# runWholeAnalysis( )

In [ ]:
    # sentidata 행의 개수별(0,30,...,150) 예측 확률 6개가 들어있는 행렬을 리턴.
    def runWholeAnalysis(self):
        cnt = 0    # count의 줄임말.
        numCorrect = np.zeros((int(self.numTraining/30) + 1, 1))    # 6X1 zero matrix.

        for j in range(0, self.numTraining+1, 30):
            self.dataSentimentTraining = self.sentidata[self.shuffle[0:j+1], 0:self.wordLimit]
            self.dataReviewTraining = self.sentidata[self.shuffle[0:j+1], -1]
            numCorrect[cnt] = 0
            for i in range(np.shape(self.dataSentimentTesting)[0]):    # 47번 for loop.
                p, n, c = self.runAnalysis(i)
                if c == 1:
                    numCorrect[cnt] += 1    # numCorrect에는 j번의 loop중 몇 개의 예측이 옳았는지 입력된다.
            numCorrect[cnt] = numCorrect[cnt] / np.shape(self.dataSentimentTesting)[0]   # 입력된 값을 j로 나눈다(평균).
            cnt += 1
        return numCorrect    # numCorrect에는 행의 개수별(0~150) 예측 확률 6개가 들어있다.

# runExperiments( ) 

In [ ]:
    # numReplicate만큼 반복해 얻은 값들(avg, avgSq, std)로 plot.
    def runExperiments(self, numReplicate):
        average = np.zeros((int(self.numTraining/30 + 1), 1))
        averageSq = np.zeros((int(self.numTraining/30 + 1), 1))

        for i in range(numReplicate):
            self.shuffle = np.arange(np.shape(self.sentidata)[0])    # 0부터 197이 들어있는 array.
            np.random.shuffle(self.shuffle)    # 그 array를 섞는다.

            self.dataSentimentTesting = self.sentidata[self.shuffle[self.numTraining+1:198], 0:self.wordLimit]    # 151부터 198, 총 47개의 행.
            self.dataReviewTesting = self.sentidata[self.shuffle[self.numTraining + 1:198], -1]    # 똑같이 47개의 행.

            correct = self.runWholeAnalysis()    # sentidata 행의 개수별(0,30,...,150) 예측 확률 6개가 들어있는 행렬.
            average = average + correct    # numReplicate만큼 반복해 average에 첨부.
            averageSq += correct * correct    # 똑같이 averageSq에 첨부.

        average = average / numReplicate    # 반복한 만큼 나눠 또다시 평균값.
        averageSq = averageSq / numReplicate
        std = np.sqrt(averageSq - average * average)

        plt.errorbar(np.arange(0, self.numTraining+1, 30), average, std)
        plt.title('Product Review Classification')
        plt.xlabel('Number of Cases')
        plt.ylabel('Percentage of Correct Classification')
        plt.show()

# calculateProbWord( )

In [ ]:
    # 인덱스 idxWord에 해당하는 단어의 긍정/부정 확률. dataSentimentTraining의 행 개수에 따라 달라진다.
    def calculateProbWord(self, idxWord):
        occurrence = [[row[idxWord]] for row in self.dataSentimentTraining]    # 중요한건 Training값이라는 것.
                                                                               # 0, 30,... 달라질 값들이다.     
        positive = np.matmul(np.transpose(occurrence), self.dataReviewTraining)    # 저번 공식과 같다. 달라진건 행렬이라는 것.
        dataNegReviewTraining = [[1-row] for row in self.dataReviewTraining]
        negative = np.matmul(np.transpose(occurrence), dataNegReviewTraining)
        positiveProb = int(positive+1) / float(positive+negative+1)    # 아마 이건 0을 방지하기 위해 1을 더한것 같다.
        negativeProb = int(negative+1) / float(positive+negative+1)
        
        return positiveProb, negativeProb

# calculateProbReview( )

In [ ]:
    # 전체 리뷰의 긍정/부정 확률. dataReviewTraining의 행 개수에 따라 달라진다.
    def calculateProbReview(self):
        numReviews = max(np.shape(self.dataReviewTraining))    # dataReviewTraining의 행 개수에 따라 달라진다.
        positive = np.sum(self.dataReviewTraining)
        negative = numReviews - positive
        positiveProb = int(positive + 1) / float(numReviews + 1)
        negativeProb = int(negative + 1) / float(numReviews + 1)
        
        return positiveProb, negativeProb

# findUsedWords( )

In [ ]:
    # idx번째 리뷰에 사용된 단어들과 단어들의 인덱스 출력.
    def findUsedWords(self, idx):
        idxUsedWords = np.where(self.dataSentimentTesting[idx] == 1)[0]
        usedWords = self.dataWord[idxUsedWords]
        
        return idxUsedWords, usedWords

# 전체 코드

In [1]:
import numpy as np
import matplotlib.pyplot as plt


class SentiAnalyzer:

    def __init__(self, sentidata, word):
        self.sentidata = sentidata    
        self.numTraining = 150   
        self.wordLimit = 1500       
        self.dataWord = word             
        print('This is a senti analyzer')

    def runAnalysis(self, idxReview):
        probLogPositive = 0
        probLogNegative = 0
        idxUsedWords, usedWords = self.findUsedWords(idxReview)

        for i in range(len(idxUsedWords)):
            idxWord = idxUsedWords[i]
            positive, negative = self.calculateProbWord(idxWord)
            probLogPositive = probLogPositive + np.log(positive)
            probLogNegative = probLogNegative + np.log(negative)

        positiveProb1, negativeProb1 = self.calculateProbReview()
        probLogPositive = probLogPositive + np.log(positiveProb1)
        probLogNegative = probLogNegative + np.log(negativeProb1)

        if self.dataReviewTesting[idxReview] == 1:
            if  probLogPositive > probLogNegative:
                correct = 1
            else:
                correct = 0
        else:
            if probLogNegative < probLogPositive:
                correct = 0
            else:
                correct = 1
        return probLogPositive, probLogNegative, correct

    def runWholeAnalysis(self):
        cnt = 0
        numCorrect = np.zeros((int(self.numTraining/30) + 1, 1))

        for j in range(0, self.numTraining+1, 30):
            self.dataSentimentTraining = self.sentidata[self.shuffle[0:j+1], 0:self.wordLimit]
            self.dataReviewTraining = self.sentidata[self.shuffle[0:j+1], -1]
            numCorrect[cnt] = 0
            for i in range(np.shape(self.dataSentimentTesting)[0]):
                p, n, c = self.runAnalysis(i)
                if c == 1:
                    numCorrect[cnt] += 1
            numCorrect[cnt] = numCorrect[cnt] / np.shape(self.dataSentimentTesting)[0]
            cnt += 1
        return numCorrect

    def runExperiments(self, numReplicate):
        average = np.zeros((int(self.numTraining/30 + 1), 1))
        averageSq = np.zeros((int(self.numTraining/30 + 1), 1))

        for i in range(numReplicate):
            self.shuffle = np.arange(np.shape(self.sentidata)[0])
            np.random.shuffle(self.shuffle)

            self.dataSentimentTesting = self.sentidata[self.shuffle[self.numTraining+1:198], 0:self.wordLimit]
            self.dataReviewTesting = self.sentidata[self.shuffle[self.numTraining + 1:198], -1]

            correct = self.runWholeAnalysis()
            average = average + correct
            averageSq += correct * correct

        average = average / numReplicate
        averageSq = averageSq / numReplicate
        std = np.sqrt(averageSq - average * average)

        plt.errorbar(np.arange(0, self.numTraining+1, 30), average, std)
        plt.title('Product Review Classification')
        plt.xlabel('Number of Cases')
        plt.ylabel('Percentage of Correct Classification')
        plt.show()

    def calculateProbWord(self, idxWord):
        occurrence = [[row[idxWord]] for row in self.dataSentimentTraining]
        positive = np.matmul(np.transpose(occurrence), self.dataReviewTraining)
        dataNegReviewTraining = [[1-row] for row in self.dataReviewTraining]
        negative = np.matmul(np.transpose(occurrence), dataNegReviewTraining)
        positiveProb = int(positive+1) / float(positive+negative+1)
        negativeProb = int(negative+1) / float(positive+negative+1)
        return positiveProb, negativeProb

    def calculateProbReview(self):
        numReviews = max(np.shape(self.dataReviewTraining))
        positive = np.sum(self.dataReviewTraining)
        negative = numReviews - positive
        positiveProb = int(positive + 1) / float(numReviews + 1)
        negativeProb = int(negative + 1) / float(numReviews + 1)
        return positiveProb, negativeProb

    def findUsedWords(self, idx):
        idxUsedWords = np.where(self.dataSentimentTesting[idx] == 1)[0]
        usedWords = self.dataWord[idxUsedWords]
        return idxUsedWords, usedWords

# 전체 코드 + 해설

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class SentiAnalyzer:
    
    # 오키 여기까지 이해 완벽하다
    # Make the method signature to accept "sentidata" and "word"
    def __init(self, sentidata, word):
        self.sentidata = sentidata # Original Dataset
        self.numTraining = 150 # number of Training
        self.wordLimit = 1500 # number of words of interests
        self.dataWord = word # list of words
        print('This is a senti analyzer')
        
    # 여기서부터가 문제.
    def runAnalysis(self, idxReview):
        probLogPositive = 0
        probLogNegative = 0
        idxUsedWords, usedWords = self.findUsedWords(idxReview) # 저번처럼 idx번째 리뷰에 사용된 word를 찾는 것.
        
        # 특정 리뷰에 나온 단어 개수만큼 loop.
        for i in range(len(idxUsedWords)):
            # 해당 리뷰에 나온 단어들의 인덱스 차례대로 입력.
            idxWord = idxUsedWords[i] 
            # idxWord 인덱스에 있는 단어의 긍정/부정적 리뷰일 확률.
            positive, negative = self.calculateProbWord(idxWord) 
            # 저번꺼랑 똑같음.
            probLogPositive = probLogPositive + np.log(positive)
            probLogNegative = probLogNegative + np.log(negative) 
            
        # 먼 Naive Bayes를 log로 바꾸고 확률 공식 어쩌구 심오한 의미가 있다.
        positiveProb1, negativeProb1 = self.calculateProbReview() # 전체 리뷰에 대한 긍정/부정 리뷰일 확률.
        probLogPositive = probLogPositive + np.log(positiveProb1)
        probLogNegative = probLogNegative + np.log(negativeProb1)
        
        # 저번꺼랑 다른 점!
        # 랜덤으로 분류된 리뷰 결과 중에 해당 리뷰가 긍정이면
        if self.dataReviewTesting[idxReview] == 1:
            if probLogPositive > probLogNegative:
                correct = 1
            else:
                correct = 0 # 이거 보면 review가 긍정으로 나오고, 실제로 분석한 결과도 긍정으로 나오면 correct 1로 출력해주는 듯.
        else:
            if probLogPositive > probLogNegative:
                correct = 0
            else:
                correct = 1 # 이건 반대로 review 결과가 부정이고 분석 결과도 부정이면 correct 1.
                
        return probLogPositive, probLogNegative, correct
        # 전체적으로 보면 저번 runAnalysis()랑 비슷한데 부수적인게 조금 바뀜. 리턴하는 것에 correct가 생기고 이상한 함수들 이용함.
        
    def runWholeAnalysis(self):
        # count 줄임말. 초기값 0.
        cnt = 0 
        # numTraining이 150이니까 6x1 zero matrix.
        numCorrect = np.zeros((int(self.numTraining/30) + 1, 1))
        
        # 0부터 150까지 30간격으로 for loop.
        for j in range(0, self.numTraining+1, 30): 
            # sentidata에서 행은 0부터 j개, 열은 0부터 1500개까지. 다하면 루프 터지니까 1500개만 한듯.
            # shuffle에 당황하지 말자 저거 시발 아무것도 아니다 (당장 지금은).
            self.dataSentimentTraining = self.sentidata[self.shuffle[0: j+1], 0:self.wordLimit]
            # 행 개수는 위랑 같고 열은 리뷰 열만 뽑아옴.
            self.dataReviewTraining = self.sentidata[self.shuffle[0:j+1],-1]
            # 초기값에 따라 numCorrect[0] = 0
            numCorrect[cnt] = 0
            # dataSentimentTesting 안나왔다고 시이이이ㅣ발
            for i in range(np.shape(self.dataSentimentTesting)[0]):
                # i번째 리뷰에 따른 긍정/부정 확률과 correct 리턴.
                p, n, c = self.runAnalysis(i)
                # correct면 numCorrect에 더하기.
                if c == 1:
                    numCorrect[cnt] += 1
            numCorrect[cnt] = numCorrect[cnt] / np.shape(self.dataSentimentTesting)[0] # Testing 몰라요 시발
            # 6번 반복해서 추가.
            cnt += 1
        return numCorrect
    
    def runExperiments(self, numReplicate): # numReplicate가 뭘까
        # numCorrect 마냥 6x1 zero matrix 싸질러놓고
        average = np.zeros((int(self.numTraining/30 + 1), 1))
        # 얘도 똑같이 싸질러놓네.
        averageSq = np.zeros((int(self.numTraining/30 + 1), 1))
        
        # iterate by the numReplicate. 아마 numReplicate가 반복하는 수인듯.
        for i in range(numReplicate):
            # 드디어 조가튼 shuffle의 정의가 나온다
            # np.shape(self.sentidata)[0]은 198.
            # 그니까 197까지 숫자 198개 array에 만듬.
            self.shuffle = np.arange(np.shape(self.sentidata)[0])
            # 이제야 섞어댄다.
            np.random.shuffle(self.shuffle)
            
            # 이제야 Testing 시벌것 나온다.
            # 왜 151부터 198까지 하는거냐... 이해좀 시켜줘라 시발 누가좀
            self.dataSentimentTesting = self.sentidata[self.shuffle[self.numTraining+1:198], 0:self.wordLimit]
            self.dataReviewTesting = self.sentidata[self.shuffle[self.numTraining+1:198], -1]
            
            # receive the correct information from runWholeAnalysis().
            # runWholeAnalysis()가 뭐 하는지도 모른다 시발 어 위에꺼네 일단 numCorrect list 가져오는듯.
            correct = self.runWholeAnalysis() # 이게 시발 끝이냐?
            
            # calculate the average by the training case sizes
            average = average + correct # 아 비어있는 average matrix에 correct인지 쓰여진 matrix를 더한다!
            
            # calculate the squared average by the training case sizes
            averageSq += correct * correct # 위에꺼랑 비슷한 논리.
            
        # finish the calculation of average
        average = average / numReplicate # 반복한만큼 나눠서 평균값.
        # finish the calculation of average squared
        averageSq = averageSq / numReplicate
        # finish the calculation of standard deviation
        std = np.sqrt(averageSq = average * average)
        
        # 먼 시발 오류 데이터를 플롯한덴다 모르겠다 이제 시발
        plt.errorbar(np.arange(0, self.numTraining+1, 30), average, std)
        plt.title('Product Review Classification')
        plt.xlabel('Number of Cases')
        plt.ylabel('Percentage of Correct Classification')
        plt.show()
    
    # idxWord 인덱스에 있는 단어의 긍정/부정적 리뷰일 확률.
    def calculateProbWord(self, idxWord):  
        # dataSentimentTraining에서 인덱스 idxWord에 해당하는 단어 나왔는지 입력된 리스트.
        occurrence = [[row[idxWord]] for row in self.dataSentimentTraining]
        # 그니까 이거 저번꺼랑 똑같이 둘다 1이면 곱해지고 그런거임. 
        # 아마 리뷰에 단어 나오고 그 리뷰 긍정적인 만큼 더해졌을듯.
        positive = np.matmul(np.transpose(occurrence), self.dataReviewTraining)
        # 부정 리뷰값이 1이 되게 바꿈.
        dataNegReviewTraining = [[1-row] for row in self.dataReviewTraining]
        # positive랑 똑같이 리뷰에 단어 나오고 그 리뷰 부정적이면 더하기. 어차피 1x1 matrix.
        negative = np.matmul(np.transpose(occurrence), dataNegReviewTraining)
        # 아마 0이 되는거 방지 위해 1 더한듯.
        positiveProb = int(positive+1) / float(positive+negative+1)
        negativeProb = int(negative+1) / float(positive+negative+1)
        
        return positiveProb, negativeProb
    
    def calculateProbReview(self):
        # 그냥 시발 len()쓰면 될것을 으렵게 폼나게 적어두노
        numReviews = max(np.shape(self.dataReviewTraining))
        # 1 다 더했음
        positive = np.sum(self.dataReviewTraining)
        negative = numReviews - positive # 알G뭔G
        # 왜 시발 1 더하는지 모르겠노
        positiveProb = int(positive + 1) / float(numReviews + 1)
        negativeProb = int(negative + 1) / float(numReviews + 1)
        return positiveProb, negativeProb        
    
    # idx번째 리뷰에 사용된 단어들과 그 단어들의 인덱스 리턴.
    def findUsedWords(self, idx):
        # idx번째 행에서 1이 있는 열 array로 만들기
        idxUsedWords = np.where(self.dataSentimentTesting[idx] == 1)[0]
        # 시발 어렵게 써서 놀랐네 걍 word.csv임.
        usedWords = self.dataWord[idxUsedWords]
        return idxUsedWords, usedWords

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt

class SentiAnalyzer:
    
    def __init__(self, sentidata, word):
        self.sentidata = sentidata
        self.numTraining = 150
        self.wordLimit = 1500
        self.dataWord = word
        print('This is a senti analyzer')
        
    def calculateProbWord(self, idxWord):
        occurrence = [[row[idxWord]] for row in self.dataSentimentTraining]
        positive = np.matmul(np.transpose(occurrence), self.dataReviewTraining)
        dataNegReviewTraining = [[1-row] for row in self.dataReviewTraining]
        negative = np.matmul(np.transpose(occurrence), dataNegReviewTraining)
        
        positiveProb = float(positive) / float(positive+negative+1)
        negativeProb =
        
    
    # idx번째 리뷰에 사용된 단어들과 단어들의 인덱스를 출력.    
    def findUsedWords(self, idx):
        idxUsedWords = np.where(self.dataSentimentTesting[idx] == 1)[0]
        usedWords = self.dataWord[idxUsedWords]
        
        return idxUsedWords, usedWords

In [ ]:
class SentiAnalyzer:
    
    def __init__():
     
    
    # idxWord 인덱스에 해당하는 단어의 긍정/부정 확률 리턴.
    def calculateProbWord():
        # 1. idxWord열에 있는 단어 출현 유무를 occurrence에 저장.
        # 2. occurrence와 긍정 리뷰 출현 유무를 곱해 positive에 저장.
        # 3. negative도 똑같이.
        # 4. pos/neg 값을 전체 리뷰 개수로 나눈다.
        # 5. 4의 결과물을 리턴.
        
    
    # idx번째 리뷰에 사용된 단어들과 단어들의 인덱스를 리턴.
    def findUsedWords():
        # 1. idx번째 리뷰에 사용된 단어들의 인덱스 저장.
        # 2. 인덱스에 따른 단어들을 dataWord에서 가져오기.
        # 3. 인덱스와 인덱스에 해당하는 단어 리턴.

In [22]:
import csv

f1 = open('word.csv', 'r', encoding='cp949')
rdr1 = csv.reader(f1)
word = []
for row in rdr1:
    word.append(row)
f1.close()
word = np.asarray(word)

f2 = open('sentidata.csv', 'r')
rdr2 = csv.reader(f2)
sentidata = []
for row in rdr2:
    sentidata.append(row)
f2.close()
sentidata = np.asarray(sentidata, dtype=np.float32)


shuffle = np.arange(np.shape(sentidata)[0])
np.random.shuffle(shuffle)


dataSentimentTesting = sentidata[shuffle[151:198], 0:1500]

np.shape(np.where(dataSentimentTesting[3] == 1))

(1, 21)

In [23]:
np.shape(np.where(dataSentimentTesting[3] == 1)[0])

(21,)

In [30]:
a = np.array([1, 2, 3])
b = np.array([3, 2, 1])
np.matmul(np.transpose(a), b)

10